In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
# imports
import pprint
import pickle
import glob
import random
from pathlib import Path
import math

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from utils import general_utils
from utils import eval_utils
import deformation.losses as def_losses

In [4]:
def compare_before_after_refinement(results_df_before, results_df_after):
    
    statistics = {"num_3d_iou_improved":0, "avg_improve_amt": 0, "avg_improve_before_2d_iou":0, "num_3d_iou_worse":0,  "avg_worse_amt":0, "avg_worse_before_2d_iou":0}
    improved_instances = []
    worse_instances = []
    comparision_df = pd.DataFrame()
    for instance in list(results_df_after["instance"]):
        if instance in list(results_df_before["instance"]):
            before_result = results_df_before[results_df_before["instance"]==instance]
            before_3d_iou = float(before_result["3d_iou"])
            #before_2d_iou = float(before_result["2d_iou"])
            after_result = results_df_after[results_df_after["instance"]==instance]
            after_3d_iou = float(after_result["3d_iou"])
            #after_2d_iou = float(after_result["2d_iou"])

            #iou_2d_change = after_2d_iou - before_2d_iou
            iou_3d_change = after_3d_iou - before_3d_iou

            if math.isnan(iou_3d_change):
                statistics["num_3d_iou_worse"] += 1
                continue
            elif iou_3d_change > 0:
                statistics["num_3d_iou_improved"] += 1
                statistics["avg_improve_amt"] += iou_3d_change
                #statistics["avg_improve_before_2d_iou"] += before_2d_iou
                improved_instances.append(instance)
            else:
                statistics["num_3d_iou_worse"] += 1
                statistics["avg_worse_amt"] += iou_3d_change
                #statistics["avg_worse_before_2d_iou"] += before_2d_iou
                worse_instances.append(instance)
            comparision_df = comparision_df.append({"instance": instance, "iou_3d_delta": iou_3d_change}, ignore_index=True)

    statistics["avg_improve_amt"] /= statistics["num_3d_iou_improved"]
    statistics["avg_improve_before_2d_iou"] /= statistics["num_3d_iou_improved"]
    statistics["avg_worse_amt"] /= statistics["num_3d_iou_worse"]
    statistics["avg_worse_before_2d_iou"] /= statistics["num_3d_iou_worse"]
            
    pprint.pprint(statistics)
    #print("example improved instances")
    #print(improved_instances[:100])
    #print("example worse instances")
    #print(worse_instances[:100])
    return comparision_df

In [5]:
def filter_eval_df_warnings(original_df, warnings_to_filter=["NaN nodes", "compute_iou_3d: gt mesh < 200 occupancies", "compute_iou_3d: rec mesh < 200 occupancies"]):
    filter_array = []
    warnings_list = original_df["eval_warnings"].tolist()
    for i in range(len(original_df)):
        warnings = warnings_list[i]
        keep=True
        for warning in warnings:
            if warning in warnings_to_filter:
                keep=False
        filter_array.append(keep)
    return original_df[filter_array]

def filter_and_sort_eval_dfs(df1, df2):
    df1 = filter_eval_df_warnings(df1)
    df2 = filter_eval_df_warnings(df2)
    df1_instances = df1["instance"].tolist()
    df2_instances = df2["instance"].tolist()
    
    df1_filters = [(True if df1_instance in df2_instances else False) for df1_instance in df1_instances]
    df1_filtered = df1[df1_filters].sort_values(by="instance")
    
    df2_filters = [(True if df2_instance in df1_instances else False) for df2_instance in df2_instances]
    df2_filtered = df2[df2_filters].sort_values(by="instance")
    
    return df1_filtered, df2_filtered

def update_2d_iou_input_to_bf(original_df, bf_2d_iou_df):
    if (original_df["instance"].tolist() != bf_2d_iou_df["instance"].tolist() ):
        raise ValueError("Instances don't match")
    original_df["2d_iou_input"] = bf_2d_iou_df["2d_iou_input"]
    return original_df
    

In [6]:

metrics = ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]
column_order = ["instance"]
for metric in metrics:
    column_order.append("{}_Before".format(metric))
    column_order.append("{}_After".format(metric))
    column_order.append("{}_Delta".format(metric))
pd.options.mode.chained_assignment = None 

def get_processed_combined_df(eval_results_path_before, eval_results_path_after, remove_low_iou=True):
    results_df_before_raw = pd.read_pickle(eval_results_path_before)
    results_df_after_raw = pd.read_pickle(eval_results_path_after)

    results_df_before, results_df_after = filter_and_sort_eval_dfs(results_df_before_raw, results_df_after_raw)
    results_df_before = results_df_before.rename(columns={"2d_iou_input":"2D_IoU", "2d_iou_multi":"MV_IoU", "3d_iou":"3D_IoU", "chamfer_L1":"CD"})
    results_df_after = results_df_after.rename(columns={"2d_iou_input":"2D_IoU", "2d_iou_multi":"MV_IoU", "3d_iou":"3D_IoU", "chamfer_L1":"CD"})
    results_df_after = results_df_after.reset_index(drop=True)
    results_df_before = results_df_before.reset_index(drop=True)
    results_df_before["CD"] = results_df_before["CD"] * 10
    results_df_after["CD"] = results_df_after["CD"] * 10
    if remove_low_iou:
        results_df_before["3D_IoU"] = results_df_before["3D_IoU"].apply(lambda x: (np.nan if x < 0.01 else x))
    results_df_after["3D_IoU"][np.isnan(results_df_before["3D_IoU"])] = np.nan
    
    metrics = ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]
    delta_df = results_df_after[metrics] / results_df_before[metrics]
    delta_df = delta_df.replace([np.inf, -np.inf], np.nan)
    
    #print("Note: filtering results from {} to {}\n".format(max(len(results_df_before_raw), len(results_df_after_raw)), len(results_df_after)))
    
    results_df_before = results_df_before.rename(columns={metric:"{}_Before".format(metric) for metric in metrics})
    results_df_before = results_df_before[["instance"]+["{}_Before".format(metric) for metric in metrics]]
    results_df_after = results_df_after.rename(columns={metric:"{}_After".format(metric)  for metric in metrics})
    results_df_after = results_df_after[["{}_After".format(metric) for metric in metrics]]
    delta_df = delta_df.rename(columns={metric:"{}_Delta".format(metric) for metric in metrics})
    delta_df = delta_df[["{}_Delta".format(metric) for metric in metrics]]
    
    combined_df = pd.concat([results_df_before, results_df_after, delta_df], axis=1)[column_order]
    
    return combined_df

def make_df_readable(original_df):
    readable_df = original_df.copy()
    readable_df.loc['mean'] = readable_df.mean()
    readable_df = readable_df.round(2)
    for column_name in original_df.columns.to_list():
        if ("_D" in column_name) or ("_C" in column_name):
            readable_df[column_name] = readable_df[column_name].apply(lambda x: "+{}%".format(round((x-1)*100)) if (x-1) > 0 else  "{}%".format(round((x-1)*100)))
        elif ("_P_" in column_name):
            readable_df[column_name] = readable_df[column_name].apply(lambda x: "{}%".format(round((x)*100)))
    return readable_df

def print_latex(input_df):
    for index, row in input_df.iterrows():
        latex_exp = " & ".join([str(x) for x in row.to_list()])
        latex_exp += " \\\\"
        latex_exp = latex_exp.replace("%", "\\%").replace("nan", "Mean")
        print(latex_exp)

In [7]:
# shapenet
shapenet_class_ids = ["02691156", "02828884", "02933112", "02958343", "03001627", "03211117", "03636649", "03691459", "04090263", "04256520", "04379243", "04401088", "04530566"]
shapenet_id_to_name = {"02691156":"Airplane", "02828884":"Bench", "02933112":"Cabinet", "02958343":"Car", "03001627":"Chair", "03211117":"Display", "03636649":"Lamp",
                       "03691459":"Speakers", "04090263":"Rifle", "04256520":"Sofa", "04379243":"Table", "04401088":"Telephone", "04530566":"Watercraft"}
pix3d_class_ids = ["bed", "bookcase", "chair", "desk", "misc", "sofa", "table","tool", "wardrobe"]
pix3d_id_to_name = {pix3d_id:pix3d_id.capitalize() for pix3d_id in pix3d_class_ids}

method_paths_dict = {
    "atlasnet":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/AtlasNet/data/pytorch_3d_render_recs/{}",
        "eval_dir_temp_after": "data/refinements/shapenet_atlasnet_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose"]
    },
    "pix2mesh":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/Pixel2Mesh/rec_files/pytorch3d_shapenet_renders/{}/rgba",
        "eval_dir_temp_after": "data/refinements/shapenet_pix2mesh_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose"]
    },
    "occnet":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/occupancy_networks/out/pytorch3d_renders/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/shapenet_occnet_refinements/{}/{}",
        "class_ids": shapenet_class_ids,
        "name_cvt_dict": shapenet_id_to_name,
        "available_settings":["gt_pose", "bf_pose", "ablation_1", "ablation_2", "ablation_3"]
    },
    "occnet_pix3d":{
        "eval_dir_temp_before": "/home/svcl-oowl/brandon/research/occupancy_networks/out/pix3d/{}/generation/meshes",
        "eval_dir_temp_after": "data/refinements/pix3d_occnet_refinements/{}/{}",
        "class_ids": pix3d_class_ids,
        "name_cvt_dict": pix3d_id_to_name,
        "available_settings":["gt_pose"]
    }
}

In [10]:
# view specific class

method = "occnet_pix3d"
#method = "occnet"
setting = "gt_pose"
class_id = "chair"

eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, True)

# filtering
#instances = ["d2f8a99bbdc387c8c5552bebbfa48bd7", "dd4da4309c12d47bc2c2c81e2232aa95", "fc7c3ccb57f65337209009cfb89d4bd"]
#combined_df = combined_df[combined_df["instance"].isin(instances)]

#display(combined_df)

print("most improved: {}".format(combined_df.sort_values("MV_IoU_Delta", ascending=False)["instance"].to_list()[:20]))
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(combined_df.sort_values("MV_IoU_Delta", ascending=False))
display(combined_df.sort_values("MV_IoU_Delta", ascending=False))

most improved: ['1359', '1422', '0649', '0581', '0863', '0859', '2868', '2475', '0770', '0652', '0647', '0966', '0661', '2931', '0927', '3313', '2969', '0744', '2944', '0672']


,instance,2D_IoU_Before,2D_IoU_After,2D_IoU_Delta,MV_IoU_Before,MV_IoU_After,MV_IoU_Delta,3D_IoU_Before,3D_IoU_After,3D_IoU_Delta,CD_Before,CD_After,CD_Delta
710,1359,0.011606,0.651577,56.143649,0.035455,0.535045,15.091015,NaN,NaN,NaN,1.467515,0.110352,0.075197
746,1422,0.043355,0.844656,19.482510,0.064338,0.667506,10.375057,NaN,NaN,NaN,6.265966,0.214588,0.034247
398,0649,0.005242,0.854642,163.043158,0.066466,0.594303,8.941394,NaN,NaN,NaN,1.570799,0.541814,0.344929
357,0581,0.011440,0.193471,16.912079,0.016047,0.137359,8.559684,NaN,NaN,NaN,2.615612,0.701249,0.268101
494,0863,0.012294,0.242720,19.742891,0.031724,0.195890,6.174787,NaN,NaN,NaN,0.816471,0.261793,0.320639
...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,2112,0.516227,0.688701,1.334105,0.605047,0.538603,0.890183,0.317151,0.188446,0.594184,0.010825,0.013450,1.242529
1437,3737,0.234926,0.240405,1.023321,0.376224,0.332129,0.882797,0.068373,0.056816,0.830974,1.050794,1.244177,1.184035
608,1155,0.433813,0.309146,0.712624,0.296968,0.259576,0.874086,0.092892,0.080599,0.867665,0.512799,0.395676,0.771600
999,2306,0.313575,0.607933,1.938713,0.314081,0.270373,0.860838,0.038838,0.057656,1.484531,0.304455,0.404873,1.329828


# Tables

In [ ]:
# table 1: all three methods side by side on shapenet with gt pose, only percentage change info
dfs_to_combine = []
for method in ["atlasnet", "pix2mesh", "occnet"]:
    quant_results_df_mean = pd.DataFrame()
    setting = "gt_pose"
    for class_id in shapenet_class_ids:
        eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
        eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

        combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

        quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
        quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]
        quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

    readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
    if len(dfs_to_combine) == 0:
        readable_quant_results_df_mean = readable_quant_results_df_mean[["instance"]+["{}_Delta".format(metric) for metric in metrics]]
    else:
        readable_quant_results_df_mean = readable_quant_results_df_mean[["{}_Delta".format(metric) for metric in metrics]]
    readable_quant_results_df_mean = readable_quant_results_df_mean.rename(columns={"{}_Delta".format(metric):"{}_{}".format(method, metric) for metric in metrics})
    dfs_to_combine.append(readable_quant_results_df_mean)
    
df_order = ["instance"]+["{}_{}".format(method, metric) for method in ["occnet", "pix2mesh", "atlasnet"]  for metric in metrics]
methods_combined_df = pd.concat(dfs_to_combine, axis=1)[df_order]
display(methods_combined_df)
print_latex(methods_combined_df)

In [ ]:
# table 2: occnet on pix3d with gt pose, only percentage change info

method = "occnet_pix3d"
quant_results_df_mean = pd.DataFrame()
setting = "gt_pose"
for class_id in pix3d_class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)

    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = class_id.capitalize()
    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

readable_quant_results_df_mean = make_df_readable(quant_results_df_mean[column_order])
readable_quant_results_df_mean = readable_quant_results_df_mean[["instance"]+["{}_Delta".format(metric) for metric in metrics]]
readable_quant_results_df_mean = readable_quant_results_df_mean.rename(columns={"{}_Delta".format(metric):"{}_{}".format(method, metric) for metric in metrics})
    
display(readable_quant_results_df_mean)
print_latex(readable_quant_results_df_mean)

In [ ]:
# table 3: occnet on shapenet bf pose, also showing percentage/amt up

method = "occnet"
setting = "bf_pose"
quant_results_df_mean = pd.DataFrame()
for class_id in shapenet_class_ids:
    eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
    eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

    combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, False)
    quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
    quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]

    for metric in ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]:
        if metric in ["CD"]:
            num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
            avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()
            num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
            avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
        else:
            num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
            avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
            num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
            avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()

        quant_result_dict_mean["{}_P_up".format(metric)] =  num_improved/(num_improved+num_worse)
        quant_result_dict_mean["{}_C_up".format(metric)] =  avg_improve_amt
        quant_result_dict_mean["{}_P_d".format(metric)] =  num_worse/(num_improved+num_worse)
        quant_result_dict_mean["{}_C_d".format(metric)] =  avg_worse_amt


    quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Before".format(metric): "{}_B".format(metric) for metric in metrics})
quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_After".format(metric): "{}_A".format(metric) for metric in metrics})
quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Delta".format(metric): "{}_C".format(metric) for metric in metrics})
readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)
df_order = ["instance"]+["{}_{}".format(metric, i) for metric in ["MV_IoU", "3D_IoU", "CD"] for i in ["P_up", "C_up", "C"]]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(readable_quant_results_df_mean[df_order])

print_latex(readable_quant_results_df_mean[df_order])


In [9]:
# Table 4: Ablation study, Shapenet, occnet, different losses

method = "occnet"
settings = ["ablation_1", "ablation_2", "ablation_3", "gt_pose"]

ablation_df = pd.DataFrame()
for setting in settings:
    print(setting)
    quant_results_df_mean = pd.DataFrame()
    for class_id in shapenet_class_ids:
        eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
        eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

        combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after, True)
        quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
        quant_result_dict_mean["instance"] = shapenet_id_to_name[class_id]
        quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)
    
    df_order = ["instance"]+["{}_Delta".format(metric) for metric in metrics]
    readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)[df_order]
    
    display(readable_quant_results_df_mean)
    ablation_mean_dict = readable_quant_results_df_mean.loc["mean"].to_dict()
    del ablation_mean_dict["instance"]
    ablation_mean_dict["ablation_setting"] = setting
    ablation_df = ablation_df.append(ablation_mean_dict, ignore_index=True)

df_order = ["ablation_setting"]+["{}_Delta".format(metric) for metric in metrics]
ablation_df = ablation_df[df_order]
display(ablation_df)
print_latex(ablation_df)


ablation_1


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+64%,-21%,-80%,+2555%
1,Bench,+32%,-5%,-57%,+867%
2,Cabinet,+10%,-6%,-25%,+288%
3,Car,+21%,-5%,-26%,+333%
4,Chair,+24%,-3%,-44%,+542%
5,Display,+16%,-7%,-28%,+416%
6,Lamp,+48%,-5%,-40%,+1153%
7,Speakers,+13%,-5%,-28%,+517%
8,Rifle,+29%,-35%,-82%,+10261%
9,Sofa,+18%,-5%,-31%,+339%


ablation_2


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+64%,+12%,-5%,-4%
1,Bench,+25%,+6%,-2%,+12%
2,Cabinet,+8%,+1%,-1%,+5%
3,Car,+19%,+4%,+5%,+4%
4,Chair,+20%,+3%,+12%,-6%
5,Display,+14%,+2%,+33%,-10%
6,Lamp,+55%,+11%,+4%,+6%
7,Speakers,+11%,+2%,+5%,+8%
8,Rifle,+74%,-6%,-13%,+129%
9,Sofa,+15%,+3%,+5%,+2%


ablation_3


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+51%,+16%,+23%,-34%
1,Bench,+18%,+6%,+12%,+2%
2,Cabinet,+7%,+3%,+3%,-9%
3,Car,+17%,+7%,+10%,-10%
4,Chair,+12%,+4%,+18%,-13%
5,Display,+13%,+5%,+54%,-23%
6,Lamp,+38%,+14%,+18%,-17%
7,Speakers,+10%,+4%,+10%,-9%
8,Rifle,+50%,+15%,+26%,-33%
9,Sofa,+12%,+4%,+7%,+50%


gt_pose


,instance,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,Airplane,+55%,+17%,+19%,-35%
1,Bench,+20%,+7%,+12%,-5%
2,Cabinet,+8%,+2%,+2%,-8%
3,Car,+18%,+7%,+11%,-10%
4,Chair,+13%,+5%,+20%,-16%
5,Display,+14%,+6%,+60%,-27%
6,Lamp,+43%,+16%,+16%,-14%
7,Speakers,+10%,+3%,+11%,-9%
8,Rifle,+62%,+13%,+18%,-24%
9,Sofa,+12%,+3%,+7%,+55%


,ablation_setting,2D_IoU_Delta,MV_IoU_Delta,3D_IoU_Delta,CD_Delta
0,ablation_1,+27%,-9%,-45%,+1628%
1,ablation_2,+29%,+4%,+5%,+16%
2,ablation_3,+22%,+8%,+21%,-10%
3,gt_pose,+25%,+8%,+23%,-10%


ablation_1 & +27\% & -9\% & -45\% & +1628\% \\
ablation_2 & +29\% & +4\% & +5\% & +16\% \\
ablation_3 & +22\% & +8\% & +21\% & -10\% \\
gt_pose & +25\% & +8\% & +23\% & -10\% \\


In [ ]:
# appendix raw tables (TODO: make sure of iou filter consistency)

for method in method_paths_dict:
    
    for setting in method_paths_dict[method]["available_settings"]:
        print("{} -- {}".format(method, setting))
        quant_results_df_mean = pd.DataFrame()
        
        for class_id in method_paths_dict[method]["class_ids"]:
            eval_results_path_before = os.path.join(method_paths_dict[method]["eval_dir_temp_before"].format(class_id), "eval_results.pkl")
            eval_results_path_after = os.path.join(method_paths_dict[method]["eval_dir_temp_after"].format(setting, class_id), "eval_results.pkl")

            combined_df = get_processed_combined_df(eval_results_path_before, eval_results_path_after)
            quant_result_dict_mean = combined_df.mean(skipna=True).to_dict()
            quant_result_dict_mean["instance"] = method_paths_dict[method]["name_cvt_dict"][class_id]

            for metric in ["2D_IoU", "MV_IoU", "3D_IoU", "CD"]:
                if metric in ["CD"]:
                    num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
                    avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()
                    num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
                    avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
                else:
                    num_improved = len(combined_df[combined_df["{}_Delta".format(metric)]>1])
                    avg_improve_amt = combined_df[combined_df["{}_Delta".format(metric)]>1]["{}_Delta".format(metric)].mean()
                    num_worse = len(combined_df[combined_df["{}_Delta".format(metric)]<1])
                    avg_worse_amt = combined_df[combined_df["{}_Delta".format(metric)]<1]["{}_Delta".format(metric)].mean()

                quant_result_dict_mean["{}_P_up".format(metric)] =  num_improved/(num_improved+num_worse)
                quant_result_dict_mean["{}_C_up".format(metric)] =  avg_improve_amt
                quant_result_dict_mean["{}_P_d".format(metric)] =  num_worse/(num_improved+num_worse)
                quant_result_dict_mean["{}_C_d".format(metric)] =  avg_worse_amt


            quant_results_df_mean = quant_results_df_mean.append(quant_result_dict_mean, ignore_index=True)

        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Before".format(metric): "{}_B".format(metric) for metric in metrics})
        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_After".format(metric): "{}_A".format(metric) for metric in metrics})
        quant_results_df_mean = quant_results_df_mean.rename(columns={"{}_Delta".format(metric): "{}_C".format(metric) for metric in metrics})
        readable_quant_results_df_mean = make_df_readable(quant_results_df_mean)
        df_order = ["instance"]+["{}_{}".format(metric, i) for metric in metrics for i in ["B", "A", "C", "P_up", "C_up", "P_d", "C_d"]]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(readable_quant_results_df_mean[df_order])
            display(readable_quant_results_df_mean[df_order].transpose())
            
        print_latex(readable_quant_results_df_mean[df_order].transpose())

    break

In [ ]:
i = 0
first = True
print("\\hline")
print(" & & Airplane & Bench & Cabinet & Car & Chair & Display & Lamp & Speakers & Rifle & Sofa & Table & Telephone & Watercraft & Mean \\\\")
for index, row in readable_quant_results_df_mean[df_order].transpose().iterrows():
    if first:
        first = False
        continue
    row_list = [str(x) for x in row.to_list()]
    if i%7==0: row_list.insert(0,"Before")
    elif i%7==1: row_list.insert(0,"After")
    elif i%7==2: row_list.insert(0,"P.C.")
    elif i%7==3: row_list.insert(0,"Prop. $\\uparrow$")
    elif i%7==4: row_list.insert(0,"Amt.  $\\uparrow$")
    elif i%7==5: row_list.insert(0,"Prop.  $\\downarrow$")
    elif i%7==6: row_list.insert(0,"Amt. $\\downarrow$")
    
    if i==0:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{2D IoU}}}")
    elif i==7:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{MV IoU}}}")
    elif i==14:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{3D IoU}}}")
    elif i==21:
        row_list.insert(0, "\\parbox[t]{3mm}{\\multirow{7}{*}{\\rotatebox[origin=c]{90}{CD}}}")
    else:
        row_list.insert(0," ")
    
    latex_exp = " & ".join(row_list)
    latex_exp += " \\\\"
    
    latex_exp = latex_exp.replace("%", "\\%").replace("nan", "Mean")
    if i % 7 == 0:
        print("\cline{1-16}")
    elif(i%7==3 or i%7==5):
        print("\cline{2-16}")
    print(latex_exp)
    i+=1
print("\\hline")